In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

In [ ]:
from descartes import PolygonPatch
from shapely.ops import unary_union, polygonize
from shapely.geometry import mapping, Polygon, Point, LineString
#from matplotlib.colors import ListedColormap

In [ ]:
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize']



# Polar FOVS

## load fovs

In [ ]:
# FOV 05
div_05 = open("CTA-ULTRA6-LaPalma-divergent_05_180.cfg")
text_05 = div_05.read()
text_05 = text_05.split("#")[1:]

tels_dict_05 = {}
for line in text_05:
    line_list = line.split("\n")
    tels_dict_05[int(line_list[0])] = {
        line_list[1].split("=")[0]: float(line_list[1].split("=")[1]),
        line_list[2].split("=")[0]: float(line_list[2].split("=")[1]),
    }

In [ ]:
# fov 2
div_2 = open("CTA-ULTRA6-LaPalma-divergent_2_180.cfg")
text_2 = div_2.read()
text_2 = text_2.split("#")[1:]

tels_dict_2 = {}
for line in text_2:
    line_list = line.split("\n")
    tels_dict_2[int(line_list[0])] = {
        line_list[1].split("=")[0]: float(line_list[1].split("=")[1]),
        line_list[2].split("=")[0]: float(line_list[2].split("=")[1]),
    }

In [ ]:
# fov 3
div_3 = open("CTA-ULTRA6-LaPalma-divergent_3_180.cfg")
text_3 = div_3.read()
text_3 = text_3.split("#")[1:]

tels_dict_3 = {}
for line in text_3:
    line_list = line.split("\n")
    tels_dict_3[int(line_list[0])] = {
        line_list[1].split("=")[0]: float(line_list[1].split("=")[1]),
        line_list[2].split("=")[0]: float(line_list[2].split("=")[1]),
    }

In [ ]:
# FOV 4
div_4 = open("CTA-ULTRA6-LaPalma-divergent_4_180.cfg")
text_4 = div_4.read()
text_4 = text_4.split("#")[1:]

tels_dict_4 = {}
for line in text_4:
    line_list = line.split("\n")
    tels_dict_4[int(line_list[0])] = {
        line_list[1].split("=")[0]: float(line_list[1].split("=")[1]),
        line_list[2].split("=")[0]: float(line_list[2].split("=")[1]),
    }

## plot fovs

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cbook as cbook

from mpl_toolkits.axisartist import Subplot
from mpl_toolkits.axisartist import SubplotHost,    ParasiteAxesAuxTrans
from mpl_toolkits.axisartist.grid_helper_curvelinear import GridHelperCurveLinear
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from matplotlib.transforms import Affine2D


def polar_stuff(fig, data):
    # PolarAxes.PolarTransform takes radian. However, we want our coordinate
    # system in degree
    tr = Affine2D().scale(np.pi/180., 1.).translate(+np.pi/2.,0) + PolarAxes.PolarTransform()

    # polar projection, which involves cycle, and also has limits in
    # its coordinates, needs a special method to find the extremes
    # (min, max of the coordinate within the view).

    # 20, 20 : number of sampling points along x, y direction
    extreme_finder = angle_helper.ExtremeFinderCycle(20, 20,
                                                     lon_cycle=360,
                                                     lat_cycle=None,
                                                     lon_minmax=None,
                                                     lat_minmax=(-90, 90),
                                                     )

    grid_locator1 = angle_helper.LocatorDMS(12)
    # Find a grid values appropriate for the coordinate (degree,
    # minute, second).

    tick_formatter1 = angle_helper.FormatterDMS()
    # And also uses an appropriate formatter.  Note that,the
    # acceptable Locator and Formatter class is a bit different than
    # that of mpl's, and you cannot directly use mpl's Locator and
    # Formatter here (but may be possible in the future).

    grid_helper = GridHelperCurveLinear(tr,
                                        extreme_finder=extreme_finder,
                                        grid_locator1=grid_locator1,
                                        tick_formatter1=tick_formatter1
                                        )

    ax1 = SubplotHost(fig, 1, 1, 1, grid_helper=grid_helper)

    # make ticklabels of right and top axis visible.
    ax1.axis["right"].major_ticklabels.set_visible(True)
    ax1.axis["top"].major_ticklabels.set_visible(True)

    # let right axis shows ticklabels for 1st coordinate (angle)
    ax1.axis["right"].get_helper().nth_coord_ticks = 0
    # let bottom axis shows ticklabels for 2nd coordinate (radius)
    ax1.axis["bottom"].get_helper().nth_coord_ticks = 1

    fig.add_subplot(ax1)

    # A parasite axes with given transform
    ax2 = ParasiteAxesAuxTrans(ax1, tr, "equal")
    # note that ax2.transData == tr + ax1.transData
    # Anything you draw in ax2 will match the ticks and grids of ax1.
    ax1.parasites.append(ax2)
    # intp = cbook.simple_linear_interpolation
    #ax2.plot(intp(np.array([0, 30]), 50),
    #         intp(np.array([10., 10.]), 50),
    #         linewidth=2.0)

    for key, value in data.items():
        if key < 5:
            continue
        theta = value['THETA']
        phi = value['PHI']
        # point.set_transform(ax2.transData)
        # transform center coordinates: 
        # circle1 = plt.Circle(
        #     (
        #         (phi-180)*(theta/np.cos((phi-180)*np.pi/180))/90*np.pi/2,
        #         #(phi-180)*(theta)/(90),
        #         -theta*np.cos((phi-180)*np.pi/180)
        #     ), 
        #     7.7/2, color='r', alpha=0.1
        # )
        circle1 = plt.Circle(
            (
                (phi-180)*np.sin(theta*np.pi/180), 
                -theta*np.cos((phi-180)*np.pi/180)
            ), 
            radius=7.7/2, 
            color="red", alpha=0.2
        )


        ax1.add_artist(circle1)
        point = ax1.scatter(phi, theta, c = "b", s = 20, zorder= 10, transform=ax2.transData)
        # ax2.scatter(phi, theta, c = "b", s = 50)
        ax2.annotate((key), (phi, theta), fontsize=15, xytext=(4, 4), textcoords='offset pixels')

    ax1.scatter(0, -20, label="MC", zorder=20, c="green")
    ax2.annotate("MC", (180, 20),  
                 fontsize=30, xytext=(-15, 4), 
                 textcoords='offset pixels', 
                 zorder=30, color="green")


    ax1.set_xlim(-10,10)
    ax1.set_ylim(-32, -8)
    ax1.set_aspect(1.)
    ax1.grid(True, zorder=0)
    ax1.set_xlabel("Azimuth in degrees", fontsize=20)
    ax1.set_ylabel("Zenith in degrees", fontsize=20)
    return fig, point

In [ ]:
tel_fake = {}
tel_fake[14] = {'PHI':190, 'THETA':30}
tel_fake[19] = {'PHI':150, 'THETA':12}


In [ ]:
fig = polar_stuff(plt.figure(figsize=(10,8)), tels_dict_4)
#plt.xlabel("test", fontsize=100)
plt.title("Pointing of MSTs: cfg 4", fontsize=15, y=1.05)
#plt.title("test",fontsize=50)

# plt.savefig("fov-scale-cfg-4_180_sphere.jpg", bbox_inches='tight')
plt.draw()
plt.show()

DISCLAIMER: this plot is almost ok. There must be some projection that I haven't taken into account when plotting the circle centers. 

# shapely

In [ ]:
# If you use the dummy, you can clearly see the superposition

dummy_test = {}
dummy_test[1] = {'THETA': 20, 'PHI': 180}
dummy_test[2] = {'THETA': 20, 'PHI': 180.1}
dummy_test[3] = {'THETA': 20, 'PHI': 179.9}


In [ ]:
# instead of tel_dict_4, you can use tel_dummy
polygons = {}
for key, value in dummy_test.items():
    polygons[key-1] = Point(value['PHI'], value['THETA']).buffer(7.7/2)

xrange = [165, 195]
yrange = [10, 30]

In [ ]:
rings = [LineString(list(pol.exterior.coords)) for pol in polygons.values()]
union = unary_union(rings)
result = {counter:geom for counter, geom in enumerate(polygonize(union))}

ori = list(polygons.values())
res = list(result.values())

In [ ]:
dict_count_overlaps = {}
for i in range(len(res)):
    dict_count_overlaps[i] = 0
    for j in range(len(ori)):
        if np.isclose(res[i].difference(ori[j]).area, 0):
            dict_count_overlaps[i] +=1
            #print(f"res_{colors[i]}, orig_{j+1}")

In [ ]:
max_multiplicity = max(dict_count_overlaps.values())

cmap = plt.cm.get_cmap('rainbow')
color_list = cmap(np.linspace(0, 1, max_multiplicity))
bounds = np.arange(max_multiplicity + 1) + 1

In [ ]:
fig = plt.figure()
gs  = mpl.gridspec.GridSpec(1, 2, width_ratios=[0.95, 0.05])

ax = plt.subplot(gs[0])
ax_cb = plt.subplot(gs[1])

fig.subplots_adjust(top=0.85)

for pol_id, pol in result.items():
    colore = dict_count_overlaps[pol_id]
    ax.add_patch(
        PolygonPatch(mapping(pol), color=color_list[colore-1])
    )

norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

cb1 = mpl.colorbar.ColorbarBase(ax_cb, 
                                norm=norm, 
                                cmap=cmap, 
                                boundaries = bounds,
                                orientation='vertical')
cb1.set_ticks(np.arange(max_multiplicity + 1) + 0.5) 
cb1.set_ticklabels(np.arange(max_multiplicity + 1) + 1)

ax.set_xlim(*xrange)
ax.set_ylim(*yrange)
ax.set_aspect(1)
plt.show()

**ori** are the original circles, whereas **res** are the single patches into which we have divided the circles. Each patch have a **.area** attribute


In [ ]:
hfov = []
for patchsky in res:
    hfov.append(patchsky.area)
    
hfov = np.array(hfov)

In [ ]:
# multiplicity associated with each patch
overlaps = np.array(list(dict_count_overlaps.values()))

In [ ]:
average_overlap = np.average(overlaps, weights=hfov)
variance = np.average((overlaps-average_overlap)**2, weights=hfov)
print(average_overlap, variance, np.sqrt(variance))